# 🚀 Comprehensive GNN-GTWR & GNN-GTVC Framework

**Implementasi Lengkap untuk Analisis Spasio-Temporal dengan Graph Neural Networks**

## 📋 Spesifikasi Framework:

### 🧠 **GNN Backbones (6 Arsitektur):**
- **GCN** (Graph Convolutional Network)
- **GAT** (Graph Attention Network)  
- **GraphSAGE** (Sample and Aggregate)
- **STGCN** (Spatial-Temporal GCN)
- **DCRNN** (Diffusion Convolutional RNN)
- **GraphWaveNet** (Graph WaveNet)

### ⚖️ **Weighting Schemes (4 Skema):**
- **Dot Product** Attention
- **Cosine Similarity** Weighting
- **Gaussian RBF** Weighting
- **MLP-based** Learnable Weighting

### 🏗️ **Model Architectures (2 Tipe):**
- **GNN-GTWR**: Geographically and Temporally Weighted Regression
- **GNN-GTVC**: Geographically and Temporally Varying Coefficients

### 🎯 **Loss Functions (2 Strategi):**
- **Fully Supervised**: Standard regression loss
- **Add Unsupervised**: + Contrastive/Reconstruction components

---

**Total Possible Configurations:** 6 × 4 × 2 × 2 = **96 Model Variants**

Notebook ini akan mengimplementasikan, melatih, dan mengevaluasi seluruh konfigurasi untuk analisis komprehensif spatial-temporal regression menggunakan Graph Neural Networks.

In [ ]:
# 📦 Import Required Libraries and Dependencies
# ==============================================
print("🚀 Loading Libraries for Comprehensive GNN Framework")
print("=" * 55)

# Core PyTorch and Scientific Computing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# PyTorch Geometric for GNN Implementation
import torch_geometric
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, TransformerConv
from torch_geometric.data import Data, Batch
from torch_geometric.utils import add_self_loops, degree

# Data Processing and Analysis
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import NearestNeighbors

# Visualization and Plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Spatial and Geospatial Processing
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr, spearmanr
import networkx as nx

# System and Utilities
import warnings
import time
import random
import os
from collections import defaultdict
from itertools import product
import json

# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Using device: {device}")

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Display versions
print(f"\n📚 Library Versions:")
print(f"   PyTorch: {torch.__version__}")
print(f"   PyTorch Geometric: {torch_geometric.__version__}")
print(f"   NumPy: {np.__version__}")
print(f"   Pandas: {pd.__version__}")

print(f"\n✅ All libraries imported successfully!")
print(f"🎯 Ready for comprehensive GNN-GTWR/GTVC implementation!")

In [ ]:
# 🌍 Load and Generate Spatial-Temporal Data
# ===========================================
print("🌍 GENERATING REALISTIC SPATIAL-TEMPORAL DATA")
print("📊 Simulating Indonesian Provincial Inflation Data")
print("=" * 55)

# Configuration parameters
N_PROVINCES = 38  # Indonesian provinces
N_TIMESTEPS = 12  # 12 months
N_FEATURES = 15   # Economic indicators

print(f"📋 Dataset Configuration:")
print(f"   Provinces: {N_PROVINCES}")
print(f"   Time Steps: {N_TIMESTEPS}")
print(f"   Features per observation: {N_FEATURES}")
print(f"   Total observations: {N_PROVINCES * N_TIMESTEPS}")

# Generate synthetic province coordinates (lat, lon)
np.random.seed(42)
province_coords = np.random.uniform(
    low=[-10, 95],   # Southern/Western bounds  
    high=[6, 141],   # Northern/Eastern bounds
    size=(N_PROVINCES, 2)
)

# Feature names (Indonesian economic indicators)
feature_names = [
    'IHK_rate',           # Consumer Price Index rate
    'PDRB_growth',        # Regional GDP growth
    'unemployment_rate',   # Unemployment rate
    'poverty_rate',       # Poverty rate
    'money_supply',       # Money supply (M2)
    'exchange_rate',      # Exchange rate (IDR/USD)
    'food_price_index',   # Food price index
    'energy_price',       # Energy price
    'population_density', # Population density
    'export_value',       # Export value
    'import_value',       # Import value
    'fiscal_balance',     # Regional fiscal balance
    'credit_growth',      # Banking credit growth
    'manufacturing_index', # Manufacturing index
    'tourism_index'       # Tourism activity index
]

# Generate realistic spatial-temporal data
def generate_spatial_temporal_data():
    """Generate realistic spatial-temporal dataset"""
    
    data = []
    
    # Base trends and patterns
    national_trend = np.sin(np.linspace(0, 2*np.pi, N_TIMESTEPS)) * 0.5
    seasonal_effect = np.cos(np.linspace(0, 4*np.pi, N_TIMESTEPS)) * 0.3
    
    for province_id in range(N_PROVINCES):
        for time_step in range(N_TIMESTEPS):
            
            # Province-specific effects
            province_effect = np.random.normal(0, 0.2)
            spatial_autocorr = 0.1 * np.sin(province_id * 0.5)
            
            # Time-varying effects
            temporal_effect = national_trend[time_step] + seasonal_effect[time_step]
            
            # Generate features with realistic correlations
            features = []
            
            # IHK rate (target-related)
            ihk_base = 3.0 + temporal_effect + province_effect
            features.append(ihk_base + np.random.normal(0, 0.5))
            
            # PDRB growth (inversely related to inflation)
            pdrb = 5.2 - 0.3 * ihk_base + np.random.normal(0, 0.8)
            features.append(pdrb)
            
            # Other economic indicators
            for i in range(2, N_FEATURES):
                # Add some correlation structure
                correlation_factor = 0.2 * features[0] + 0.1 * features[1]
                feature_val = correlation_factor + np.random.normal(0, 1.0)
                features.append(feature_val)
            
            # Target variable (inflation rate)
            inflation_base = 0.4 * features[0] + 0.2 * features[1] - 0.1 * features[2]
            spatial_spillover = spatial_autocorr * 0.3
            noise = np.random.normal(0, 0.3)
            
            target = inflation_base + spatial_spillover + noise
            
            # Store observation
            data.append({
                'province_id': province_id,
                'time_step': time_step,
                'lat': province_coords[province_id, 0],
                'lon': province_coords[province_id, 1],
                **{feature_names[i]: features[i] for i in range(N_FEATURES)},
                'inflation_rate': target
            })
    
    return pd.DataFrame(data)

# Generate the dataset
df = generate_spatial_temporal_data()

print(f"\n✅ Dataset Generated Successfully!")
print(f"   Shape: {df.shape}")
print(f"   Columns: {list(df.columns)}")
print(f"   Inflation rate range: [{df['inflation_rate'].min():.3f}, {df['inflation_rate'].max():.3f}]")

# Display sample data
print(f"\n📊 Sample Data Preview:")
print(df.head(10))

# Basic statistics
print(f"\n📈 Target Variable Statistics:")
print(df['inflation_rate'].describe())

print(f"\n🎯 Data ready for graph construction and GNN training!")

In [ ]:
# 🧠 Implement GNN Backbone Models  
# =================================
print("🧠 IMPLEMENTING GNN BACKBONE ARCHITECTURES")
print("🎯 6 State-of-the-Art Architectures with PyTorch Geometric")
print("=" * 60)

class GCNBackbone(nn.Module):
    """Graph Convolutional Network backbone"""
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.1):
        super(GCNBackbone, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        
        # First layer
        self.convs.append(GCNConv(input_dim, hidden_dim))
        self.bns.append(nn.BatchNorm1d(hidden_dim))
        
        # Hidden layers
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
            self.bns.append(nn.BatchNorm1d(hidden_dim))
            
    def forward(self, x, edge_index, edge_weight=None):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index, edge_weight)
            x = self.bns[i](x)
            if i < len(self.convs) - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x

class GATBackbone(nn.Module):
    """Graph Attention Network backbone"""
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, heads=4, dropout=0.1):
        super(GATBackbone, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        
        # First layer
        self.convs.append(GATConv(input_dim, hidden_dim // heads, heads=heads, dropout=dropout))
        self.bns.append(nn.BatchNorm1d(hidden_dim))
        
        # Hidden layers
        for _ in range(num_layers - 1):
            self.convs.append(GATConv(hidden_dim, hidden_dim // heads, heads=heads, dropout=dropout))
            self.bns.append(nn.BatchNorm1d(hidden_dim))
            
    def forward(self, x, edge_index, edge_weight=None):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            x = self.bns[i](x)
            if i < len(self.convs) - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x

class GraphSAGEBackbone(nn.Module):
    """GraphSAGE backbone"""  
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.1):
        super(GraphSAGEBackbone, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        
        # First layer
        self.convs.append(SAGEConv(input_dim, hidden_dim))
        self.bns.append(nn.BatchNorm1d(hidden_dim))
        
        # Hidden layers
        for _ in range(num_layers - 1):
            self.convs.append(SAGEConv(hidden_dim, hidden_dim))
            self.bns.append(nn.BatchNorm1d(hidden_dim))
            
    def forward(self, x, edge_index, edge_weight=None):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            x = self.bns[i](x)
            if i < len(self.convs) - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        return x

class STGCNBackbone(nn.Module):
    """Spatial-Temporal Graph Convolutional Network backbone"""
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.1):
        super(STGCNBackbone, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        
        # Spatial convolutions
        self.spatial_convs = nn.ModuleList()
        self.spatial_convs.append(GCNConv(input_dim, hidden_dim))
        for _ in range(num_layers - 1):
            self.spatial_convs.append(GCNConv(hidden_dim, hidden_dim))
            
        # Temporal convolutions  
        self.temporal_conv = nn.Conv1d(hidden_dim, hidden_dim, 3, padding=1)
        
        # Batch normalization
        self.bns = nn.ModuleList()
        for _ in range(num_layers):
            self.bns.append(nn.BatchNorm1d(hidden_dim))
            
    def forward(self, x, edge_index, edge_weight=None):
        # Spatial processing
        for i, conv in enumerate(self.spatial_convs):
            x = conv(x, edge_index, edge_weight)
            x = self.bns[i](x)
            if i < len(self.spatial_convs) - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Temporal processing (simplified)
        x_temp = x.unsqueeze(0).transpose(1, 2)  # [1, hidden_dim, nodes]
        x_temp = self.temporal_conv(x_temp)
        x = x_temp.transpose(1, 2).squeeze(0)    # [nodes, hidden_dim]
        
        return x

class DCRNNBackbone(nn.Module):
    """Diffusion Convolutional RNN backbone"""
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.1):
        super(DCRNNBackbone, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        
        # GRU cells for temporal modeling
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, 
                         batch_first=True, dropout=dropout if num_layers > 1 else 0)
        
        # Spatial diffusion layers
        self.spatial_convs = nn.ModuleList()
        for _ in range(2):  # Multiple diffusion steps
            self.spatial_convs.append(GCNConv(hidden_dim, hidden_dim))
            
        self.bn = nn.BatchNorm1d(hidden_dim)
        
    def forward(self, x, edge_index, edge_weight=None):
        # Temporal processing via GRU
        x_seq = x.unsqueeze(1)  # [nodes, 1, features] - treat as sequence
        h, _ = self.gru(x_seq)
        h = h.squeeze(1)  # [nodes, hidden_dim]
        
        # Spatial diffusion
        for conv in self.spatial_convs:
            h = conv(h, edge_index, edge_weight)
            h = F.relu(h)
            h = F.dropout(h, p=self.dropout, training=self.training)
            
        h = self.bn(h)
        return h

class GraphWaveNetBackbone(nn.Module):
    """Graph WaveNet backbone (simplified)"""
    def __init__(self, input_dim, hidden_dim=64, num_layers=4, dropout=0.1):
        super(GraphWaveNetBackbone, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        
        # Dilated convolutions for temporal
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()
        self.skip_convs = nn.ModuleList()
        
        # Spatial graph convolutions
        self.spatial_convs = nn.ModuleList()
        
        for i in range(num_layers):
            in_dim = input_dim if i == 0 else hidden_dim
            
            self.filter_convs.append(nn.Linear(in_dim, hidden_dim))
            self.gate_convs.append(nn.Linear(in_dim, hidden_dim))
            self.skip_convs.append(nn.Linear(hidden_dim, hidden_dim))
            self.spatial_convs.append(GCNConv(hidden_dim, hidden_dim))
    
    def forward(self, x, edge_index, edge_weight=None):
        skip_connections = []
        
        for i in range(self.num_layers):
            # Gated temporal convolution
            filter_out = torch.tanh(self.filter_convs[i](x))
            gate_out = torch.sigmoid(self.gate_convs[i](x))
            x_gated = filter_out * gate_out
            
            # Spatial convolution
            x_spatial = self.spatial_convs[i](x_gated, edge_index, edge_weight)
            x_spatial = F.relu(x_spatial)
            
            # Skip connection
            skip = self.skip_convs[i](x_spatial)
            skip_connections.append(skip)
            
            x = x_spatial
        
        # Combine skip connections
        output = torch.stack(skip_connections, dim=0).sum(dim=0)
        return output

# Registry of all backbone architectures
BACKBONE_REGISTRY = {
    'GCN': GCNBackbone,
    'GAT': GATBackbone,
    'GraphSAGE': GraphSAGEBackbone,
    'STGCN': STGCNBackbone,
    'DCRNN': DCRNNBackbone,
    'GraphWaveNet': GraphWaveNetBackbone
}

print(f"\n✅ GNN BACKBONE ARCHITECTURES IMPLEMENTED:")
for i, name in enumerate(BACKBONE_REGISTRY.keys(), 1):
    print(f"   {i}. {name}")

print(f"\n🚀 All {len(BACKBONE_REGISTRY)} backbone architectures ready!")
print(f"🎯 Each supports customizable hidden dimensions and layers!")

In [ ]:
# ⚖️ Implement Weighting Schemes
# ===============================
print("⚖️ IMPLEMENTING SPATIAL-TEMPORAL WEIGHTING SCHEMES")
print("🎯 4 Advanced Weighting Mechanisms")
print("=" * 50)

class DotProductWeighting(nn.Module):
    """Dot product-based attention weighting"""
    def __init__(self, feature_dim, temperature=1.0):
        super(DotProductWeighting, self).__init__()
        self.feature_dim = feature_dim
        self.temperature = temperature
        self.scale = np.sqrt(feature_dim)
        
    def forward(self, query, key, value=None):
        """
        Args:
            query: [N, D] node features
            key: [N, D] neighbor features  
            value: [N, D] optional value features
        Returns:
            weights: [N] attention weights
        """
        if value is None:
            value = key
            
        # Compute dot product attention
        scores = torch.sum(query * key, dim=1) / (self.scale * self.temperature)
        weights = torch.softmax(scores, dim=0)
        
        return weights

class CosineWeighting(nn.Module):
    """Cosine similarity-based weighting"""
    def __init__(self, feature_dim, eps=1e-8):
        super(CosineWeighting, self).__init__()
        self.eps = eps
        
    def forward(self, query, key, value=None):
        """Compute cosine similarity weights"""
        if value is None:
            value = key
            
        # Normalize vectors
        query_norm = F.normalize(query, p=2, dim=1, eps=self.eps)
        key_norm = F.normalize(key, p=2, dim=1, eps=self.eps)
        
        # Cosine similarity
        cosine_sim = torch.sum(query_norm * key_norm, dim=1)
        
        # Convert to positive weights [0, 1]
        weights = (cosine_sim + 1) / 2
        weights = weights / (torch.sum(weights) + self.eps)  # Normalize
        
        return weights

class GaussianWeighting(nn.Module):
    """Gaussian RBF-based weighting"""
    def __init__(self, feature_dim, initial_sigma=1.0, learnable=True):
        super(GaussianWeighting, self).__init__()
        if learnable:
            self.sigma = nn.Parameter(torch.tensor(initial_sigma))
        else:
            self.register_buffer('sigma', torch.tensor(initial_sigma))
            
    def forward(self, query, key, value=None):
        """Compute Gaussian RBF weights"""
        if value is None:
            value = key
            
        # Euclidean distance
        diff = query - key
        squared_dist = torch.sum(diff * diff, dim=1)
        
        # Gaussian RBF
        weights = torch.exp(-squared_dist / (2 * self.sigma ** 2))
        weights = weights / (torch.sum(weights) + 1e-8)  # Normalize
        
        return weights

class MLPWeighting(nn.Module):
    """MLP-based learnable weighting"""
    def __init__(self, feature_dim, hidden_dim=64, num_layers=2, dropout=0.1):
        super(MLPWeighting, self).__init__()
        
        layers = []
        
        # Input layer
        layers.append(nn.Linear(feature_dim * 2, hidden_dim))  # Concatenated features
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))
        
        # Hidden layers
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            
        # Output layer
        layers.append(nn.Linear(hidden_dim, 1))
        layers.append(nn.Sigmoid())
        
        self.mlp = nn.Sequential(*layers)
        
    def forward(self, query, key, value=None):
        """Compute MLP-based weights"""
        if value is None:
            value = key
            
        # Concatenate query and key features
        combined = torch.cat([query, key], dim=1)
        
        # MLP prediction
        weights = self.mlp(combined).squeeze(1)
        weights = weights / (torch.sum(weights) + 1e-8)  # Normalize
        
        return weights

class AdaptiveWeighting(nn.Module):
    """Adaptive weighting that combines multiple schemes"""
    def __init__(self, feature_dim, schemes=['dot_product', 'cosine', 'gaussian'], hidden_dim=32):
        super(AdaptiveWeighting, self).__init__()
        
        self.schemes = nn.ModuleDict()
        
        if 'dot_product' in schemes:
            self.schemes['dot_product'] = DotProductWeighting(feature_dim)
        if 'cosine' in schemes:
            self.schemes['cosine'] = CosineWeighting(feature_dim)
        if 'gaussian' in schemes:
            self.schemes['gaussian'] = GaussianWeighting(feature_dim)
        if 'mlp' in schemes:
            self.schemes['mlp'] = MLPWeighting(feature_dim, hidden_dim)
            
        # Combination weights
        self.combination_weights = nn.Parameter(torch.ones(len(schemes)) / len(schemes))
        
    def forward(self, query, key, value=None):
        """Compute adaptive combination of weighting schemes"""
        if value is None:
            value = key
            
        scheme_weights = []
        
        for scheme_name, scheme_module in self.schemes.items():
            weights = scheme_module(query, key, value)
            scheme_weights.append(weights)
            
        # Stack and combine
        stacked_weights = torch.stack(scheme_weights, dim=1)  # [N, num_schemes]
        combination_weights = F.softmax(self.combination_weights, dim=0)
        
        # Weighted combination
        final_weights = torch.sum(stacked_weights * combination_weights, dim=1)
        final_weights = final_weights / (torch.sum(final_weights) + 1e-8)
        
        return final_weights

# Registry of weighting schemes
WEIGHTING_REGISTRY = {
    'dot_product': DotProductWeighting,
    'cosine': CosineWeighting,
    'gaussian': GaussianWeighting,
    'mlp': MLPWeighting
}

print(f"\n✅ WEIGHTING SCHEMES IMPLEMENTED:")
for i, name in enumerate(WEIGHTING_REGISTRY.keys(), 1):
    print(f"   {i}. {name.replace('_', ' ').title()}")

print(f"\n🚀 All {len(WEIGHTING_REGISTRY)} weighting schemes ready!")
print(f"🎯 Each supports different similarity/distance computations!")

In [ ]:
# 🏗️ Build GNN-GTWR Model Architecture
# =====================================
print("🏗️ IMPLEMENTING GNN-GTWR ARCHITECTURE")
print("📊 Geographically and Temporally Weighted Regression")
print("=" * 55)

class GNN_GTWR(nn.Module):
    """
    GNN-based Geographically and Temporally Weighted Regression
    Combines GNN backbone with spatial-temporal weighting for regression
    """
    def __init__(self, backbone_name, weighting_scheme, input_dim, 
                 hidden_dim=64, output_dim=1, num_layers=2, dropout=0.1):
        super(GNN_GTWR, self).__init__()
        
        self.backbone_name = backbone_name
        self.weighting_scheme_name = weighting_scheme
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # Initialize GNN backbone
        BackboneClass = BACKBONE_REGISTRY[backbone_name]
        self.backbone = BackboneClass(input_dim, hidden_dim, num_layers, dropout)
        
        # Initialize weighting scheme
        WeightingClass = WEIGHTING_REGISTRY[weighting_scheme]
        self.weighting = WeightingClass(hidden_dim)
        
        # Local regression layers
        self.local_regressor = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, output_dim)
        )
        
        # Global context layer
        self.global_context = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2)
        )
        
        # Coefficient prediction for interpretability
        self.coeff_predictor = nn.Linear(hidden_dim, input_dim)
        
        # Final combination layer
        self.final_layer = nn.Linear(hidden_dim // 2 + output_dim, output_dim)
        
    def forward(self, x, edge_index, edge_weight=None, return_coeffs=False, return_weights=False):
        """
        Forward pass for GNN-GTWR
        
        Args:
            x: Node features [N, input_dim]
            edge_index: Graph edges [2, E]
            edge_weight: Edge weights [E] (optional)
            return_coeffs: Whether to return local coefficients
            return_weights: Whether to return attention weights
        """
        batch_size = x.size(0)
        
        # Extract features using GNN backbone
        h = self.backbone(x, edge_index, edge_weight)  # [N, hidden_dim]
        
        # Compute spatial-temporal weights
        if edge_index.size(1) > 0:
            row, col = edge_index
            
            # Compute attention weights between connected nodes
            query_features = h[row]  # Source nodes
            key_features = h[col]    # Target nodes
            
            spatial_weights = self.weighting(query_features, key_features)
            
            # Aggregate weighted neighbor information
            weighted_messages = spatial_weights.unsqueeze(1) * key_features
            
            # Aggregate by summing messages for each node
            aggregated = torch.zeros_like(h)
            aggregated.index_add_(0, row, weighted_messages)
            
            # Combine original features with aggregated messages
            h_weighted = h + 0.5 * aggregated  # Residual connection
        else:
            h_weighted = h
            spatial_weights = torch.ones(0, device=x.device)
        
        # Local regression prediction
        local_pred = self.local_regressor(h_weighted)  # [N, output_dim]
        
        # Global context
        global_context = self.global_context(h_weighted)  # [N, hidden_dim//2]
        
        # Combine local and global information
        combined = torch.cat([global_context, local_pred], dim=1)
        final_pred = self.final_layer(combined)  # [N, output_dim]
        
        results = {'predictions': final_pred}
        
        if return_coeffs:
            local_coeffs = self.coeff_predictor(h_weighted)  # [N, input_dim]
            results['coefficients'] = local_coeffs
            
        if return_weights:
            results['spatial_weights'] = spatial_weights
            results['node_embeddings'] = h_weighted
            
        return results if (return_coeffs or return_weights) else final_pred

class GTWRLoss(nn.Module):
    """Custom loss function for GTWR with spatial smoothness regularization"""
    def __init__(self, alpha_spatial=0.1, alpha_coeff=0.01):
        super(GTWRLoss, self).__init__()
        self.alpha_spatial = alpha_spatial
        self.alpha_coeff = alpha_coeff
        self.mse_loss = nn.MSELoss()
        
    def forward(self, predictions, targets, edge_index=None, coefficients=None):
        """
        Compute GTWR loss with regularization terms
        
        Args:
            predictions: Model predictions [N]
            targets: Target values [N]
            edge_index: Graph edges for spatial smoothness [2, E]
            coefficients: Local coefficients for regularization [N, input_dim]
        """
        # Base regression loss
        base_loss = self.mse_loss(predictions, targets)
        
        total_loss = base_loss
        loss_dict = {'regression': base_loss.item()}
        
        # Spatial smoothness regularization
        if edge_index is not None and edge_index.size(1) > 0:
            row, col = edge_index
            pred_diff = predictions[row] - predictions[col]
            spatial_loss = torch.mean(pred_diff ** 2)
            total_loss += self.alpha_spatial * spatial_loss
            loss_dict['spatial'] = spatial_loss.item()
        
        # Coefficient regularization
        if coefficients is not None:
            coeff_loss = torch.mean(coefficients ** 2)
            total_loss += self.alpha_coeff * coeff_loss
            loss_dict['coefficient'] = coeff_loss.item()
            
        loss_dict['total'] = total_loss.item()
        
        return total_loss, loss_dict

print(f"\n✅ GNN-GTWR ARCHITECTURE IMPLEMENTED:")
print(f"   🏗️  Main Model: GNN_GTWR")
print(f"   🎯 Custom Loss: GTWRLoss (with spatial regularization)")
print(f"   📊 Features: Local regression + Global context")
print(f"   🔍 Interpretability: Local coefficient prediction")
print(f"   ⚖️  Supports all {len(BACKBONE_REGISTRY)} backbones × {len(WEIGHTING_REGISTRY)} weighting schemes")

print(f"\n🚀 GNN-GTWR ready for training!")
print(f"🎯 Total possible configurations: {len(BACKBONE_REGISTRY) * len(WEIGHTING_REGISTRY)}")

In [ ]:
# 🏛️ Build GNN-GTVC Model Architecture  
# =====================================
print("🏛️ IMPLEMENTING GNN-GTVC ARCHITECTURE")
print("📈 Geographically and Temporally Varying Coefficients")
print("=" * 55)

class GNN_GTVC(nn.Module):
    """
    GNN-based Geographically and Temporally Varying Coefficients
    Models spatially and temporally varying relationships between predictors and response
    """
    def __init__(self, backbone_name, weighting_scheme, input_dim,
                 hidden_dim=64, output_dim=1, num_layers=2, dropout=0.1):
        super(GNN_GTVC, self).__init__()
        
        self.backbone_name = backbone_name
        self.weighting_scheme_name = weighting_scheme
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # Initialize GNN backbone
        BackboneClass = BACKBONE_REGISTRY[backbone_name]
        self.backbone = BackboneClass(input_dim, hidden_dim, num_layers, dropout)
        
        # Initialize weighting scheme
        WeightingClass = WEIGHTING_REGISTRY[weighting_scheme]
        self.weighting = WeightingClass(hidden_dim)
        
        # Varying coefficient generators
        self.coeff_generator = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, input_dim)  # One coefficient per input feature
        )
        
        # Intercept generator
        self.intercept_generator = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, output_dim)
        )
        
        # Non-linear transformation option
        self.nonlinear_transform = nn.Sequential(
            nn.Linear(input_dim, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, input_dim)
        )
        
        # Spatial-temporal effect modulator
        self.st_modulator = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.Tanh(),  # Bounded modulation
            nn.Linear(hidden_dim // 2, 1)
        )
        
    def forward(self, x, edge_index, edge_weight=None, return_components=False):
        """
        Forward pass for GNN-GTVC
        
        Args:
            x: Node features [N, input_dim]
            edge_index: Graph edges [2, E]  
            edge_weight: Edge weights [E] (optional)
            return_components: Whether to return coefficient components
        """
        batch_size = x.size(0)
        
        # Extract spatial-temporal representations using GNN backbone
        h = self.backbone(x, edge_index, edge_weight)  # [N, hidden_dim]
        
        # Apply spatial weighting if edges exist
        if edge_index.size(1) > 0:
            row, col = edge_index
            
            # Compute spatial attention weights
            query_features = h[row]
            key_features = h[col]
            spatial_weights = self.weighting(query_features, key_features)
            
            # Apply spatial weighting through message passing
            weighted_messages = spatial_weights.unsqueeze(1) * key_features
            
            # Aggregate messages
            h_aggregated = torch.zeros_like(h)
            h_aggregated.index_add_(0, row, weighted_messages)
            
            # Combine with residual connection
            h_weighted = h + 0.3 * h_aggregated
        else:
            h_weighted = h
            spatial_weights = torch.ones(0, device=x.device)
        
        # Generate spatially and temporally varying coefficients
        local_coeffs = self.coeff_generator(h_weighted)  # [N, input_dim]
        
        # Generate varying intercepts
        local_intercepts = self.intercept_generator(h_weighted)  # [N, output_dim]
        
        # Optional non-linear feature transformation
        x_transformed = x + 0.1 * self.nonlinear_transform(x)
        
        # Spatial-temporal modulation factor
        st_modulation = self.st_modulator(h_weighted)  # [N, 1]
        
        # Compute predictions using varying coefficients
        # y_i = sum(coeff_i * x_i) * modulation_i + intercept_i
        coefficient_effects = torch.sum(local_coeffs * x_transformed, dim=1, keepdim=True)
        modulated_effects = coefficient_effects * torch.sigmoid(st_modulation)
        predictions = modulated_effects + local_intercepts
        
        if return_components:
            return {
                'predictions': predictions,
                'coefficients': local_coeffs,
                'intercepts': local_intercepts,
                'modulation': st_modulation,
                'spatial_weights': spatial_weights,
                'node_embeddings': h_weighted
            }
        
        return predictions

class GTVCLoss(nn.Module):
    """Custom loss function for GTVC with coefficient regularization"""
    def __init__(self, alpha_coeff_smooth=0.1, alpha_coeff_sparse=0.01, alpha_intercept=0.01):
        super(GTVCLoss, self).__init__()
        self.alpha_coeff_smooth = alpha_coeff_smooth
        self.alpha_coeff_sparse = alpha_coeff_sparse  
        self.alpha_intercept = alpha_intercept
        self.mse_loss = nn.MSELoss()
        
    def forward(self, predictions, targets, edge_index=None, coefficients=None, intercepts=None):
        """
        Compute GTVC loss with coefficient regularization
        
        Args:
            predictions: Model predictions [N]
            targets: Target values [N]
            edge_index: Graph edges for coefficient smoothness [2, E]
            coefficients: Local coefficients [N, input_dim]
            intercepts: Local intercepts [N, 1]
        """
        # Base regression loss
        base_loss = self.mse_loss(predictions, targets)
        
        total_loss = base_loss
        loss_dict = {'regression': base_loss.item()}
        
        # Coefficient spatial smoothness
        if edge_index is not None and edge_index.size(1) > 0 and coefficients is not None:
            row, col = edge_index
            coeff_diff = coefficients[row] - coefficients[col]  # [E, input_dim]
            coeff_smooth_loss = torch.mean(torch.sum(coeff_diff ** 2, dim=1))
            total_loss += self.alpha_coeff_smooth * coeff_smooth_loss
            loss_dict['coeff_smooth'] = coeff_smooth_loss.item()
        
        # Coefficient sparsity (L1 regularization)
        if coefficients is not None:
            coeff_sparse_loss = torch.mean(torch.abs(coefficients))
            total_loss += self.alpha_coeff_sparse * coeff_sparse_loss
            loss_dict['coeff_sparse'] = coeff_sparse_loss.item()
            
        # Intercept regularization
        if intercepts is not None:
            intercept_loss = torch.mean(intercepts ** 2)
            total_loss += self.alpha_intercept * intercept_loss
            loss_dict['intercept'] = intercept_loss.item()
            
        loss_dict['total'] = total_loss.item()
        
        return total_loss, loss_dict

print(f"\n✅ GNN-GTVC ARCHITECTURE IMPLEMENTED:")
print(f"   🏛️  Main Model: GNN_GTVC")  
print(f"   🎯 Custom Loss: GTVCLoss (with coefficient regularization)")
print(f"   📊 Features: Varying coefficients + Intercepts + Modulation")
print(f"   🔍 Interpretability: Local coefficient interpretation")
print(f"   ⚖️  Supports all {len(BACKBONE_REGISTRY)} backbones × {len(WEIGHTING_REGISTRY)} weighting schemes")

print(f"\n🚀 GNN-GTVC ready for training!")
print(f"🎯 Both GNN-GTWR and GNN-GTVC architectures complete!")
print(f"📊 Total model combinations: {len(BACKBONE_REGISTRY) * len(WEIGHTING_REGISTRY) * 2}")

In [ ]:
# 🎯 Define Loss Functions (Supervised vs Unsupervised)
# =====================================================
print("🎯 IMPLEMENTING ADVANCED LOSS FUNCTIONS")
print("⚡ Supervised vs Supervised + Unsupervised Components")
print("=" * 60)

class SupervisedLoss(nn.Module):
    """Standard fully supervised loss for labeled data only"""
    def __init__(self, loss_type='mse', reduction='mean'):
        super(SupervisedLoss, self).__init__()
        self.loss_type = loss_type
        self.reduction = reduction
        
        if loss_type == 'mse':
            self.criterion = nn.MSELoss(reduction=reduction)
        elif loss_type == 'mae':
            self.criterion = nn.L1Loss(reduction=reduction)
        elif loss_type == 'huber':
            self.criterion = nn.SmoothL1Loss(reduction=reduction)
        elif loss_type == 'mape':
            self.criterion = self._mape_loss
        else:
            raise ValueError(f"Unknown loss type: {loss_type}")
    
    def _mape_loss(self, predictions, targets):
        """Mean Absolute Percentage Error"""
        return torch.mean(torch.abs((targets - predictions) / (targets + 1e-8)))
    
    def forward(self, predictions, targets, mask=None):
        """
        Compute supervised loss
        
        Args:
            predictions: Model predictions [N]
            targets: Ground truth targets [N]
            mask: Boolean mask for labeled samples [N]
        """
        if mask is not None:
            predictions = predictions[mask]
            targets = targets[mask]
        
        loss = self.criterion(predictions, targets)
        return loss, {'supervised': loss.item()}

class SemiSupervisedLoss(nn.Module):
    """Semi-supervised loss combining supervised + unsupervised components"""
    def __init__(self, supervised_loss_type='mse', 
                 alpha_consistency=0.1, alpha_contrastive=0.05, 
                 alpha_reconstruction=0.02, temperature=0.5):
        super(SemiSupervisedLoss, self).__init__()
        
        # Supervised component
        self.supervised_loss = SupervisedLoss(supervised_loss_type)
        
        # Unsupervised component weights
        self.alpha_consistency = alpha_consistency
        self.alpha_contrastive = alpha_contrastive  
        self.alpha_reconstruction = alpha_reconstruction
        self.temperature = temperature
        
    def consistency_loss(self, predictions1, predictions2, mask=None):
        """Consistency loss between different augmentations/views"""
        if mask is not None:
            predictions1 = predictions1[mask]
            predictions2 = predictions2[mask]
            
        return F.mse_loss(predictions1, predictions2)
    
    def contrastive_loss(self, embeddings, edge_index, mask=None):
        """Contrastive loss for learning better representations"""
        if edge_index.size(1) == 0:
            return torch.tensor(0.0, device=embeddings.device)
            
        row, col = edge_index
        
        if mask is not None:
            # Filter edges to only include nodes in mask
            valid_edges = mask[row] & mask[col]
            row = row[valid_edges]
            col = col[valid_edges]
            
            if len(row) == 0:
                return torch.tensor(0.0, device=embeddings.device)
        
        # Compute similarities
        pos_sim = F.cosine_similarity(embeddings[row], embeddings[col], dim=1)
        pos_sim = pos_sim / self.temperature
        
        # Negative sampling (random pairs)
        neg_indices = torch.randperm(embeddings.size(0), device=embeddings.device)[:len(row)]
        neg_sim = F.cosine_similarity(embeddings[row], embeddings[neg_indices], dim=1)
        neg_sim = neg_sim / self.temperature
        
        # Contrastive loss (InfoNCE style)
        pos_exp = torch.exp(pos_sim)
        neg_exp = torch.exp(neg_sim)
        
        loss = -torch.mean(torch.log(pos_exp / (pos_exp + neg_exp + 1e-8)))
        return loss
    
    def reconstruction_loss(self, original_features, reconstructed_features, mask=None):
        """Reconstruction loss for autoencoder-style regularization"""
        if mask is not None:
            original_features = original_features[mask]
            reconstructed_features = reconstructed_features[mask]
            
        return F.mse_loss(reconstructed_features, original_features)
    
    def forward(self, predictions, targets, labeled_mask, 
                edge_index=None, embeddings=None, reconstructed_features=None,
                predictions_aug=None, original_features=None):
        """
        Compute semi-supervised loss with multiple components
        
        Args:
            predictions: Model predictions [N]
            targets: Ground truth targets [N]
            labeled_mask: Mask for labeled samples [N]
            edge_index: Graph edges [2, E]
            embeddings: Node embeddings for contrastive loss [N, D]
            reconstructed_features: Reconstructed input features [N, input_dim]
            predictions_aug: Predictions from augmented inputs [N]
            original_features: Original input features [N, input_dim]
        """
        # Supervised loss (only on labeled data)
        sup_loss, sup_dict = self.supervised_loss(predictions, targets, labeled_mask)
        
        total_loss = sup_loss
        loss_dict = sup_dict.copy()
        
        # Consistency loss (if augmented predictions provided)
        if predictions_aug is not None:
            consistency_loss = self.consistency_loss(predictions, predictions_aug, labeled_mask)
            total_loss += self.alpha_consistency * consistency_loss
            loss_dict['consistency'] = consistency_loss.item()
        
        # Contrastive loss (on all nodes - labeled and unlabeled)
        if embeddings is not None and edge_index is not None:
            contrastive_loss = self.contrastive_loss(embeddings, edge_index)
            total_loss += self.alpha_contrastive * contrastive_loss
            loss_dict['contrastive'] = contrastive_loss.item()
        
        # Reconstruction loss (on all nodes)
        if reconstructed_features is not None and original_features is not None:
            reconstruction_loss = self.reconstruction_loss(original_features, reconstructed_features)
            total_loss += self.alpha_reconstruction * reconstruction_loss
            loss_dict['reconstruction'] = reconstruction_loss.item()
        
        loss_dict['total'] = total_loss.item()
        
        return total_loss, loss_dict

class AdversarialLoss(nn.Module):
    """Adversarial training loss for robustness"""
    def __init__(self, base_loss_type='mse', epsilon=0.01, alpha=0.1):
        super(AdversarialLoss, self).__init__()
        self.base_loss = SupervisedLoss(base_loss_type)
        self.epsilon = epsilon
        self.alpha = alpha
    
    def forward(self, model, x, targets, mask, edge_index=None):
        """
        Compute adversarial loss with FGSM-style perturbations
        
        Args:
            model: The GNN model
            x: Input features [N, input_dim]
            targets: Ground truth targets [N]
            mask: Boolean mask for labeled samples [N]
            edge_index: Graph edges [2, E]
        """
        # Standard prediction
        predictions = model(x, edge_index)
        clean_loss, clean_dict = self.base_loss(predictions, targets, mask)
        
        # Generate adversarial perturbations
        x_adv = x.clone().detach().requires_grad_(True)
        predictions_adv = model(x_adv, edge_index)
        adv_loss_temp, _ = self.base_loss(predictions_adv, targets, mask)
        
        # Compute gradients
        grad = torch.autograd.grad(adv_loss_temp, x_adv, retain_graph=True)[0]
        
        # FGSM perturbation
        perturbation = self.epsilon * torch.sign(grad)
        x_perturbed = x + perturbation
        
        # Prediction on perturbed input
        predictions_perturbed = model(x_perturbed, edge_index)
        adv_loss, adv_dict = self.base_loss(predictions_perturbed, targets, mask)
        
        # Combined loss
        total_loss = clean_loss + self.alpha * adv_loss
        
        loss_dict = {
            'clean': clean_loss.item(),
            'adversarial': adv_loss.item(),
            'total': total_loss.item()
        }
        
        return total_loss, loss_dict

# Loss function registry
LOSS_REGISTRY = {
    'supervised': SupervisedLoss,
    'semi_supervised': SemiSupervisedLoss,
    'adversarial': AdversarialLoss
}

print(f"\n✅ ADVANCED LOSS FUNCTIONS IMPLEMENTED:")
print(f"   1. 🎯 SupervisedLoss (MSE/MAE/Huber/MAPE)")
print(f"   2. 🔄 SemiSupervisedLoss (+ Consistency + Contrastive + Reconstruction)")
print(f"   3. ⚔️  AdversarialLoss (+ FGSM perturbations)")

print(f"\n🔥 LOSS FUNCTION COMPONENTS:")
print(f"   📊 Supervised: Standard regression loss on labeled data")
print(f"   🔄 Consistency: Agreement between different augmentations")
print(f"   🤝 Contrastive: Similar nodes have similar embeddings")
print(f"   🔧 Reconstruction: Autoencoder-style feature reconstruction")
print(f"   ⚔️  Adversarial: Robustness against input perturbations")

print(f"\n🚀 All loss functions ready for training!")
print(f"🎯 Supports both fully supervised and semi-supervised learning!")

In [ ]:
# 🕸️ Graph Construction & Data Preparation
# ========================================
print("🕸️ CONSTRUCTING SPATIAL-TEMPORAL GRAPH")
print("🌐 Building Graph Structure for GNN Training")
print("=" * 50)

def create_spatial_adjacency(coords, k_neighbors=5, distance_threshold=None):
    """Create spatial adjacency matrix based on geographic coordinates"""
    n_nodes = len(coords)
    
    # Compute pairwise distances
    distances = squareform(pdist(coords, metric='euclidean'))
    
    adjacency = np.zeros((n_nodes, n_nodes))
    
    if distance_threshold is not None:
        # Threshold-based adjacency
        adjacency = (distances <= distance_threshold).astype(float)
    else:
        # k-NN based adjacency
        for i in range(n_nodes):
            # Find k nearest neighbors (excluding self)
            nearest_indices = np.argsort(distances[i])[1:k_neighbors+1]
            adjacency[i, nearest_indices] = 1
            adjacency[nearest_indices, i] = 1  # Make symmetric
    
    # Remove self-connections
    np.fill_diagonal(adjacency, 0)
    
    return adjacency

def create_temporal_adjacency(n_provinces, n_timesteps, temporal_window=2):
    """Create temporal adjacency matrix connecting same province across time"""
    n_total = n_provinces * n_timesteps
    adjacency = np.zeros((n_total, n_total))
    
    for province in range(n_provinces):
        for t in range(n_timesteps):
            current_idx = province * n_timesteps + t
            
            # Connect to previous and next time steps within window
            for dt in range(-temporal_window, temporal_window + 1):
                if dt == 0:
                    continue
                    
                neighbor_t = t + dt
                if 0 <= neighbor_t < n_timesteps:
                    neighbor_idx = province * n_timesteps + neighbor_t
                    # Weight inversely proportional to temporal distance
                    weight = 1.0 / (abs(dt) + 1)
                    adjacency[current_idx, neighbor_idx] = weight
    
    return adjacency

def create_feature_similarity_adjacency(features, similarity_threshold=0.7):
    """Create adjacency based on feature similarity"""
    # Compute feature correlations
    correlations = np.corrcoef(features)
    correlations = np.nan_to_num(correlations, 0)
    
    # Threshold-based adjacency
    adjacency = (np.abs(correlations) >= similarity_threshold).astype(float)
    np.fill_diagonal(adjacency, 0)  # Remove self-connections
    
    return adjacency

def create_hybrid_graph(df, spatial_weight=0.4, temporal_weight=0.3, feature_weight=0.3):
    """Create hybrid graph combining spatial, temporal, and feature similarities"""
    
    n_total = len(df)
    
    # Extract coordinates and features
    coords = df[['lat', 'lon']].values
    feature_cols = [col for col in df.columns if col not in ['province_id', 'time_step', 'lat', 'lon', 'inflation_rate']]
    features = df[feature_cols].values
    
    print(f"📊 Graph Construction Parameters:")
    print(f"   Total nodes: {n_total}")
    print(f"   Spatial weight: {spatial_weight}")
    print(f"   Temporal weight: {temporal_weight}")
    print(f"   Feature weight: {feature_weight}")
    
    # 1. Spatial adjacency (province-level)
    unique_coords = []
    coord_to_idx = {}
    for i, (province_id, time_step) in enumerate(zip(df['province_id'], df['time_step'])):
        if province_id not in coord_to_idx:
            coord_to_idx[province_id] = len(unique_coords)
            unique_coords.append(coords[i])
    
    unique_coords = np.array(unique_coords)
    spatial_adj_provinces = create_spatial_adjacency(unique_coords, k_neighbors=6)
    
    # Expand spatial adjacency to all time steps
    spatial_adj = np.zeros((n_total, n_total))
    for i in range(n_total):
        for j in range(n_total):
            province_i = df.iloc[i]['province_id']
            province_j = df.iloc[j]['province_id']
            
            if province_i in coord_to_idx and province_j in coord_to_idx:
                spatial_idx_i = coord_to_idx[province_i]
                spatial_idx_j = coord_to_idx[province_j]
                spatial_adj[i, j] = spatial_adj_provinces[spatial_idx_i, spatial_idx_j]
    
    # 2. Temporal adjacency
    temporal_adj = create_temporal_adjacency(N_PROVINCES, N_TIMESTEPS, temporal_window=2)
    
    # 3. Feature similarity adjacency
    feature_adj = create_feature_similarity_adjacency(features, similarity_threshold=0.6)
    
    # 4. Combine adjacencies
    combined_adj = (spatial_weight * spatial_adj + 
                   temporal_weight * temporal_adj + 
                   feature_weight * feature_adj)
    
    # Normalize and threshold
    combined_adj = combined_adj / np.max(combined_adj)
    combined_adj = (combined_adj > 0.1).astype(float) * combined_adj
    
    # Convert to edge_index format
    edge_indices = np.where(combined_adj > 0)
    edge_index = torch.tensor(np.stack(edge_indices), dtype=torch.long)
    edge_weights = torch.tensor(combined_adj[edge_indices], dtype=torch.float32)
    
    print(f"   Edges created: {edge_index.size(1)}")
    print(f"   Average degree: {edge_index.size(1) / n_total:.2f}")
    
    return edge_index, edge_weights, combined_adj

# Create graph structure
edge_index, edge_weights, adjacency_matrix = create_hybrid_graph(df)

# Prepare features and targets
feature_cols = [col for col in df.columns if col not in ['province_id', 'time_step', 'lat', 'lon', 'inflation_rate']]
X = df[feature_cols].values
y = df['inflation_rate'].values

# Standardize features
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# Convert to tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

# Create train/test split (temporal split for realism)
train_ratio = 0.8
n_train = int(len(df) * train_ratio)

# Temporal split: train on earlier time periods
train_indices = []
test_indices = []

for province in range(N_PROVINCES):
    province_data = df[df['province_id'] == province].index.tolist()
    province_data.sort()  # Ensure temporal order
    
    n_province_train = int(len(province_data) * train_ratio)
    train_indices.extend(province_data[:n_province_train])
    test_indices.extend(province_data[n_province_train:])

train_indices = torch.tensor(train_indices, dtype=torch.long)
test_indices = torch.tensor(test_indices, dtype=torch.long)

# Create masks
train_mask = torch.zeros(len(df), dtype=torch.bool)
test_mask = torch.zeros(len(df), dtype=torch.bool)
train_mask[train_indices] = True
test_mask[test_indices] = True

print(f"\n✅ GRAPH AND DATA PREPARATION COMPLETE:")
print(f"   📊 Total samples: {len(df)}")
print(f"   🚂 Training samples: {train_mask.sum().item()}")
print(f"   🧪 Testing samples: {test_mask.sum().item()}")
print(f"   🕸️  Graph edges: {edge_index.size(1)}")
print(f"   📈 Features: {X_tensor.shape[1]}")

print(f"\n🚀 Ready for comprehensive model training!")
print(f"🎯 Graph structure supports semi-supervised learning!")

In [ ]:
# 🚂 Train Models with Different Configurations
# ==============================================
print("🚂 COMPREHENSIVE MODEL TRAINING FRAMEWORK")
print("⚡ Training All Backbone × Weighting × Loss Combinations")
print("=" * 60)

class ComprehensiveTrainer:
    """Comprehensive training framework for all model configurations"""
    
    def __init__(self, X, y, edge_index, edge_weights, train_mask, test_mask, device='cpu'):
        self.X = X.to(device)
        self.y = y.to(device) 
        self.edge_index = edge_index.to(device)
        self.edge_weights = edge_weights.to(device)
        self.train_mask = train_mask.to(device)
        self.test_mask = test_mask.to(device)
        self.device = device
        
        self.results = []
        
    def create_model(self, model_type, backbone, weighting, input_dim, hidden_dim=64):
        """Create model instance"""
        if model_type == 'GTWR':
            model = GNN_GTWR(backbone, weighting, input_dim, hidden_dim)
        elif model_type == 'GTVC':
            model = GNN_GTVC(backbone, weighting, input_dim, hidden_dim)
        else:
            raise ValueError(f"Unknown model type: {model_type}")
            
        return model.to(self.device)
    
    def create_loss_function(self, loss_type):
        """Create loss function instance"""
        if loss_type == 'supervised':
            return SupervisedLoss()
        elif loss_type == 'semi_supervised':
            return SemiSupervisedLoss()
        else:
            raise ValueError(f"Unknown loss type: {loss_type}")
    
    def train_single_model(self, model, loss_fn, epochs=200, lr=0.01, weight_decay=1e-4, 
                          patience=30, verbose=False):
        """Train a single model configuration"""
        
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                        factor=0.5, patience=15, verbose=False)
        
        best_train_loss = float('inf')
        best_test_r2 = -float('inf')
        patience_counter = 0
        
        train_losses = []
        test_r2_scores = []
        
        model.train()
        
        for epoch in range(epochs):
            optimizer.zero_grad()
            
            # Forward pass
            if isinstance(model, GNN_GTVC):
                results = model(self.X, self.edge_index, return_components=True)
                predictions = results['predictions'].squeeze()
                coefficients = results.get('coefficients')
                embeddings = results.get('node_embeddings')
            else:
                results = model(self.X, self.edge_index, return_coeffs=True, return_weights=True)
                if isinstance(results, dict):
                    predictions = results['predictions'].squeeze()
                    coefficients = results.get('coefficients')
                    embeddings = results.get('node_embeddings')
                else:
                    predictions = results.squeeze()
                    coefficients = None
                    embeddings = None
            
            # Compute loss
            if isinstance(loss_fn, SemiSupervisedLoss):
                loss, loss_dict = loss_fn(
                    predictions, self.y, self.train_mask,
                    edge_index=self.edge_index,
                    embeddings=embeddings
                )
            else:
                loss, loss_dict = loss_fn(predictions, self.y, self.train_mask)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            # Evaluation
            model.eval()
            with torch.no_grad():
                if isinstance(model, GNN_GTVC):
                    eval_results = model(self.X, self.edge_index)
                else:
                    eval_results = model(self.X, self.edge_index)
                    
                if isinstance(eval_results, dict):
                    eval_predictions = eval_results['predictions'].squeeze()
                else:
                    eval_predictions = eval_results.squeeze()
                
                # Training metrics
                train_pred = eval_predictions[self.train_mask]
                train_true = self.y[self.train_mask]
                train_r2 = r2_score(train_true.cpu().numpy(), train_pred.cpu().numpy())
                
                # Test metrics
                test_pred = eval_predictions[self.test_mask]
                test_true = self.y[self.test_mask]
                test_r2 = r2_score(test_true.cpu().numpy(), test_pred.cpu().numpy())
                
            model.train()
            
            # Track metrics
            train_losses.append(loss.item())
            test_r2_scores.append(test_r2)
            
            # Learning rate scheduling
            scheduler.step(loss)
            
            # Early stopping
            if loss.item() < best_train_loss:
                best_train_loss = loss.item()
                patience_counter = 0
            else:
                patience_counter += 1
                
            if test_r2 > best_test_r2:
                best_test_r2 = test_r2
                
            if patience_counter >= patience:
                if verbose:
                    print(f"    Early stopping at epoch {epoch}")
                break
                
            if verbose and epoch % 50 == 0:
                print(f"    Epoch {epoch}: Loss={loss.item():.4f}, Test R²={test_r2:.4f}")
        
        return {
            'final_train_loss': best_train_loss,
            'best_test_r2': best_test_r2,
            'train_losses': train_losses,
            'test_r2_scores': test_r2_scores,
            'epochs_trained': epoch + 1
        }
    
    def evaluate_model(self, model):
        """Comprehensive model evaluation"""
        model.eval()
        with torch.no_grad():
            if isinstance(model, GNN_GTVC):
                results = model(self.X, self.edge_index, return_components=True)
                predictions = results['predictions'].squeeze()
            else:
                predictions = model(self.X, self.edge_index).squeeze()
            
            # Training metrics
            train_pred = predictions[self.train_mask].cpu().numpy()
            train_true = self.y[self.train_mask].cpu().numpy()
            
            train_r2 = r2_score(train_true, train_pred)
            train_rmse = np.sqrt(mean_squared_error(train_true, train_pred))
            train_mae = mean_absolute_error(train_true, train_pred)
            
            # Test metrics
            test_pred = predictions[self.test_mask].cpu().numpy()
            test_true = self.y[self.test_mask].cpu().numpy()
            
            test_r2 = r2_score(test_true, test_pred)
            test_rmse = np.sqrt(mean_squared_error(test_true, test_pred))
            test_mae = mean_absolute_error(test_true, test_pred)
            
            return {
                'train_r2': train_r2,
                'train_rmse': train_rmse,
                'train_mae': train_mae,
                'test_r2': test_r2,
                'test_rmse': test_rmse,
                'test_mae': test_mae
            }
    
    def run_experiment(self, model_type, backbone, weighting, loss_type, 
                      hidden_dim=64, epochs=200, verbose=False):
        """Run single experiment configuration"""
        
        config_name = f"{model_type}-{backbone}-{weighting}-{loss_type}"
        
        if verbose:
            print(f"\n🔥 Training: {config_name}")
        
        try:
            # Create model and loss function
            model = self.create_model(model_type, backbone, weighting, 
                                    self.X.shape[1], hidden_dim)
            loss_fn = self.create_loss_function(loss_type)
            
            # Train model
            training_results = self.train_single_model(model, loss_fn, epochs=epochs, 
                                                     verbose=verbose)
            
            # Evaluate model
            eval_results = self.evaluate_model(model)
            
            # Combine results
            result = {
                'model_type': model_type,
                'backbone': backbone,
                'weighting': weighting,
                'loss_type': loss_type,
                'hidden_dim': hidden_dim,
                'status': 'success',
                **training_results,
                **eval_results
            }
            
            if verbose:
                print(f"    ✅ Success: Train R²={eval_results['train_r2']:.4f}, "
                     f"Test R²={eval_results['test_r2']:.4f}")
            
        except Exception as e:
            result = {
                'model_type': model_type,
                'backbone': backbone,
                'weighting': weighting,
                'loss_type': loss_type,
                'hidden_dim': hidden_dim,
                'status': 'failed',
                'error': str(e),
                'train_r2': np.nan,
                'test_r2': np.nan
            }
            
            if verbose:
                print(f"    ❌ Failed: {str(e)}")
        
        self.results.append(result)
        return result

# Initialize trainer
trainer = ComprehensiveTrainer(
    X_tensor, y_tensor, edge_index, edge_weights, 
    train_mask, test_mask, device
)

print(f"\n✅ COMPREHENSIVE TRAINER INITIALIZED:")
print(f"   🖥️  Device: {device}")
print(f"   📊 Training samples: {train_mask.sum().item()}")
print(f"   🧪 Testing samples: {test_mask.sum().item()}")
print(f"   🕸️  Graph edges: {edge_index.size(1)}")

print(f"\n🚀 Ready for comprehensive experiments!")
print(f"🎯 Framework supports all model configurations!")

In [ ]:
# 🧪 Model Evaluation and Comparison
# ==================================
print("🧪 COMPREHENSIVE MODEL EVALUATION & COMPARISON")
print("📊 Running Key Model Configurations")
print("=" * 55)

def run_key_experiments():
    """Run key experimental configurations for demonstration"""
    
    # Select key configurations to test
    key_configs = [
        # GNN-GTWR configurations
        ('GTWR', 'GCN', 'dot_product', 'supervised'),
        ('GTWR', 'GCN', 'cosine', 'semi_supervised'),
        ('GTWR', 'GAT', 'gaussian', 'supervised'),
        ('GTWR', 'GraphSAGE', 'mlp', 'semi_supervised'),
        
        # GNN-GTVC configurations  
        ('GTVC', 'GCN', 'dot_product', 'supervised'),
        ('GTVC', 'GAT', 'cosine', 'semi_supervised'),
        ('GTVC', 'STGCN', 'gaussian', 'supervised'),
        ('GTVC', 'GraphSAGE', 'mlp', 'supervised'),
        
        # Advanced backbones
        ('GTWR', 'DCRNN', 'cosine', 'supervised'),
        ('GTVC', 'GraphWaveNet', 'dot_product', 'semi_supervised'),
    ]
    
    print(f"🎯 Running {len(key_configs)} key configurations:")
    print("=" * 50)
    
    results = []
    
    for i, (model_type, backbone, weighting, loss_type) in enumerate(key_configs):
        print(f"\n[{i+1}/{len(key_configs)}] {model_type}-{backbone}-{weighting}-{loss_type}")
        
        result = trainer.run_experiment(
            model_type=model_type,
            backbone=backbone, 
            weighting=weighting,
            loss_type=loss_type,
            hidden_dim=64,
            epochs=150,
            verbose=True
        )
        
        results.append(result)
    
    return results

# Run key experiments
experiment_results = run_key_experiments()

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(experiment_results)

print(f"\n📊 EXPERIMENT RESULTS SUMMARY:")
print("=" * 40)

# Filter successful results
successful_results = results_df[results_df['status'] == 'success'].copy()

if len(successful_results) > 0:
    print(f"✅ Successful runs: {len(successful_results)}/{len(results_df)}")
    print(f"📈 Best Test R²: {successful_results['test_r2'].max():.6f}")
    print(f"📊 Mean Test R²: {successful_results['test_r2'].mean():.6f}")
    print(f"📉 Std Test R²: {successful_results['test_r2'].std():.6f}")
    
    # Best configuration
    best_idx = successful_results['test_r2'].idxmax()
    best_config = successful_results.loc[best_idx]
    
    print(f"\n🏆 BEST CONFIGURATION:")
    print(f"   Model: {best_config['model_type']}")
    print(f"   Backbone: {best_config['backbone']}")  
    print(f"   Weighting: {best_config['weighting']}")
    print(f"   Loss: {best_config['loss_type']}")
    print(f"   Test R²: {best_config['test_r2']:.6f}")
    print(f"   Train R²: {best_config['train_r2']:.6f}")
    print(f"   Test RMSE: {best_config['test_rmse']:.6f}")
    
    # Performance by model type
    print(f"\n📊 PERFORMANCE BY MODEL TYPE:")
    model_performance = successful_results.groupby('model_type')['test_r2'].agg(['mean', 'std', 'count'])
    for model_type, stats in model_performance.iterrows():
        print(f"   {model_type}: R²={stats['mean']:.4f}±{stats['std']:.4f} (n={stats['count']})")
    
    # Performance by backbone
    print(f"\n🧠 PERFORMANCE BY BACKBONE:")
    backbone_performance = successful_results.groupby('backbone')['test_r2'].agg(['mean', 'std', 'count'])
    for backbone, stats in backbone_performance.iterrows():
        print(f"   {backbone}: R²={stats['mean']:.4f}±{stats['std']:.4f} (n={stats['count']})")
    
    # Performance by weighting scheme
    print(f"\n⚖️ PERFORMANCE BY WEIGHTING SCHEME:")
    weighting_performance = successful_results.groupby('weighting')['test_r2'].agg(['mean', 'std', 'count'])
    for weighting, stats in weighting_performance.iterrows():
        print(f"   {weighting}: R²={stats['mean']:.4f}±{stats['std']:.4f} (n={stats['count']})")
    
    # Performance by loss type
    print(f"\n🎯 PERFORMANCE BY LOSS TYPE:")
    loss_performance = successful_results.groupby('loss_type')['test_r2'].agg(['mean', 'std', 'count'])
    for loss_type, stats in loss_performance.iterrows():
        print(f"   {loss_type}: R²={stats['mean']:.4f}±{stats['std']:.4f} (n={stats['count']})")

else:
    print("❌ No successful experiments completed")

# Save results
results_df.to_csv('GNN_Comprehensive_Experiment_Results.csv', index=False)
print(f"\n💾 Results saved to: GNN_Comprehensive_Experiment_Results.csv")

print(f"\n🎉 COMPREHENSIVE EVALUATION COMPLETE!")
print(f"🚀 Results ready for visualization and analysis!")

In [ ]:
# 📊 Spatial-Temporal Weight Visualization
# ========================================
print("📊 CREATING SPATIAL-TEMPORAL WEIGHT VISUALIZATIONS")
print("🎨 Advanced Visualizations for Thesis Integration")
print("=" * 55)

def visualize_model_performance(results_df):
    """Create comprehensive performance visualizations"""
    
    successful_results = results_df[results_df['status'] == 'success'].copy()
    
    if len(successful_results) == 0:
        print("❌ No successful results to visualize")
        return
    
    # Set up plotting style
    plt.style.use('default')
    sns.set_palette("husl")
    
    # Create comprehensive performance plots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('GNN-GTWR/GTVC Comprehensive Performance Analysis', 
                 fontsize=16, fontweight='bold')
    
    # 1. Performance by Model Type
    model_perf = successful_results.groupby('model_type')['test_r2'].agg(['mean', 'std']).reset_index()
    bars1 = axes[0,0].bar(model_perf['model_type'], model_perf['mean'], 
                         yerr=model_perf['std'], capsize=5, 
                         color=['skyblue', 'lightcoral'])
    axes[0,0].set_title('Test R² by Model Type')
    axes[0,0].set_ylabel('Test R²')
    axes[0,0].set_ylim(0, 1)
    
    # Add value labels
    for bar, val in zip(bars1, model_perf['mean']):
        axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                      f'{val:.3f}', ha='center', va='bottom')
    
    # 2. Performance by Backbone
    backbone_perf = successful_results.groupby('backbone')['test_r2'].agg(['mean', 'std']).reset_index()
    bars2 = axes[0,1].bar(backbone_perf['backbone'], backbone_perf['mean'],
                         yerr=backbone_perf['std'], capsize=5, color='lightgreen')
    axes[0,1].set_title('Test R² by GNN Backbone')
    axes[0,1].set_ylabel('Test R²')
    axes[0,1].tick_params(axis='x', rotation=45)
    axes[0,1].set_ylim(0, 1)
    
    # 3. Performance by Weighting Scheme
    weighting_perf = successful_results.groupby('weighting')['test_r2'].agg(['mean', 'std']).reset_index()
    bars3 = axes[0,2].bar(weighting_perf['weighting'], weighting_perf['mean'],
                         yerr=weighting_perf['std'], capsize=5, color='gold')
    axes[0,2].set_title('Test R² by Weighting Scheme')
    axes[0,2].set_ylabel('Test R²')
    axes[0,2].tick_params(axis='x', rotation=45)
    axes[0,2].set_ylim(0, 1)
    
    # 4. Performance by Loss Type
    loss_perf = successful_results.groupby('loss_type')['test_r2'].agg(['mean', 'std']).reset_index()
    bars4 = axes[1,0].bar(loss_perf['loss_type'], loss_perf['mean'],
                         yerr=loss_perf['std'], capsize=5, color='plum')
    axes[1,0].set_title('Test R² by Loss Function')
    axes[1,0].set_ylabel('Test R²')
    axes[1,0].set_ylim(0, 1)
    
    # 5. Train vs Test R² Scatter
    axes[1,1].scatter(successful_results['train_r2'], successful_results['test_r2'], 
                     alpha=0.7, s=60)
    axes[1,1].plot([0, 1], [0, 1], 'r--', alpha=0.7, label='Perfect Generalization')
    axes[1,1].set_xlabel('Train R²')
    axes[1,1].set_ylabel('Test R²')
    axes[1,1].set_title('Train vs Test R² (Generalization)')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    # 6. Top Configurations
    top_configs = successful_results.nlargest(5, 'test_r2')
    config_names = [f"{row['model_type']}-{row['backbone']}-{row['weighting']}" 
                   for _, row in top_configs.iterrows()]
    
    bars6 = axes[1,2].bar(range(len(top_configs)), top_configs['test_r2'], 
                         color=plt.cm.viridis(np.linspace(0, 1, len(top_configs))))
    axes[1,2].set_title('Top 5 Model Configurations')
    axes[1,2].set_ylabel('Test R²')
    axes[1,2].set_xlabel('Configuration Rank')
    axes[1,2].set_xticks(range(len(top_configs)))
    axes[1,2].set_xticklabels([f'#{i+1}' for i in range(len(top_configs))])
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars6, top_configs['test_r2'])):
        axes[1,2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                      f'{val:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('GNN_Comprehensive_Performance_Analysis.png', 
                dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    
    return fig

def create_detailed_heatmap(results_df):
    """Create detailed performance heatmap"""
    
    successful_results = results_df[results_df['status'] == 'success'].copy()
    
    if len(successful_results) == 0:
        return None
    
    # Create pivot table for heatmap
    pivot_data = successful_results.pivot_table(
        values='test_r2',
        index=['model_type', 'backbone'], 
        columns=['weighting', 'loss_type'],
        aggfunc='mean'
    )
    
    plt.figure(figsize=(14, 10))
    
    # Create heatmap
    sns.heatmap(pivot_data, annot=True, fmt='.3f', cmap='viridis',
                cbar_kws={'label': 'Test R²'}, linewidths=0.5)
    
    plt.title('Comprehensive Model Performance Heatmap\n(Test R² Scores)', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Weighting Scheme & Loss Function', fontsize=12)
    plt.ylabel('Model Type & GNN Backbone', fontsize=12)
    
    plt.tight_layout()
    plt.savefig('GNN_Performance_Heatmap.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    
    return pivot_data

def visualize_spatial_patterns(adjacency_matrix, df):
    """Visualize spatial connectivity patterns"""
    
    plt.figure(figsize=(12, 8))
    
    # Show adjacency matrix
    plt.subplot(2, 2, 1)
    im = plt.imshow(adjacency_matrix[:100, :100], cmap='Blues', aspect='auto')
    plt.title('Spatial-Temporal Adjacency Matrix (Sample)')
    plt.xlabel('Node Index')
    plt.ylabel('Node Index')
    plt.colorbar(im, shrink=0.8)
    
    # Degree distribution
    plt.subplot(2, 2, 2)
    degrees = np.sum(adjacency_matrix > 0, axis=1)
    plt.hist(degrees, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    plt.title('Node Degree Distribution')
    plt.xlabel('Degree')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    
    # Geographic distribution of provinces
    plt.subplot(2, 2, 3)
    unique_coords = df.groupby('province_id')[['lat', 'lon']].first()
    plt.scatter(unique_coords['lon'], unique_coords['lat'], 
               alpha=0.7, s=60, c='red')
    plt.title('Provincial Geographic Distribution')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.grid(True, alpha=0.3)
    
    # Temporal connectivity strength
    plt.subplot(2, 2, 4)
    temporal_strengths = []
    for t in range(N_TIMESTEPS-1):
        strength = 0
        for p in range(N_PROVINCES):
            idx1 = p * N_TIMESTEPS + t
            idx2 = p * N_TIMESTEPS + t + 1
            if idx1 < len(adjacency_matrix) and idx2 < len(adjacency_matrix):
                strength += adjacency_matrix[idx1, idx2]
        temporal_strengths.append(strength / N_PROVINCES)
    
    plt.plot(range(len(temporal_strengths)), temporal_strengths, 
             marker='o', linewidth=2, markersize=6)
    plt.title('Temporal Connectivity Strength Over Time')
    plt.xlabel('Time Step')
    plt.ylabel('Average Connectivity')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('Spatial_Temporal_Patterns.png', dpi=300, bbox_inches='tight')
    plt.show()

# Create visualizations
if 'results_df' in locals() and len(results_df) > 0:
    print("🎨 Creating performance visualizations...")
    
    # Main performance plots
    perf_fig = visualize_model_performance(results_df)
    
    # Detailed heatmap
    print("🔥 Creating detailed heatmap...")
    heatmap_data = create_detailed_heatmap(results_df)
    
    # Spatial patterns
    print("🗺️ Creating spatial pattern visualizations...")
    visualize_spatial_patterns(adjacency_matrix, df)
    
    print("✅ All visualizations created successfully!")
    
else:
    print("⚠️ No results available for visualization")

print(f"\n🎉 VISUALIZATION SUITE COMPLETE!")
print(f"📊 Generated publication-quality figures for thesis integration!")
print(f"🎯 Ready for Bab4TA1.tex integration!")

In [ ]:
# 🏆 Performance Analysis Across Backbones and Weighting Schemes
# ===============================================================
print("🏆 COMPREHENSIVE PERFORMANCE ANALYSIS & INTERPRETATION")
print("📈 Statistical Analysis & Thesis-Ready Results")
print("=" * 65)

def comprehensive_statistical_analysis(results_df):
    """Perform comprehensive statistical analysis of results"""
    
    successful_results = results_df[results_df['status'] == 'success'].copy()
    
    if len(successful_results) == 0:
        print("❌ No successful results for analysis")
        return
    
    print("📊 STATISTICAL ANALYSIS RESULTS:")
    print("=" * 40)
    
    # Overall performance statistics
    print(f"✅ OVERALL PERFORMANCE:")
    print(f"   Mean Test R²: {successful_results['test_r2'].mean():.6f}")
    print(f"   Std Test R²:  {successful_results['test_r2'].std():.6f}")
    print(f"   Min Test R²:  {successful_results['test_r2'].min():.6f}")
    print(f"   Max Test R²:  {successful_results['test_r2'].max():.6f}")
    print(f"   Median Test R²: {successful_results['test_r2'].median():.6f}")
    
    # Significance tests between model types
    gtwr_r2 = successful_results[successful_results['model_type'] == 'GTWR']['test_r2']
    gtvc_r2 = successful_results[successful_results['model_type'] == 'GTVC']['test_r2']
    
    if len(gtwr_r2) > 0 and len(gtvc_r2) > 0:
        from scipy.stats import ttest_ind
        t_stat, p_val = ttest_ind(gtwr_r2, gtvc_r2)
        print(f"\n🔬 STATISTICAL SIGNIFICANCE (GTWR vs GTVC):")
        print(f"   T-statistic: {t_stat:.4f}")
        print(f"   P-value: {p_val:.4f}")
        print(f"   Significant: {'Yes' if p_val < 0.05 else 'No'}")
    
    # Ranking analysis
    print(f"\n🏅 TOP 5 CONFIGURATIONS:")
    top_5 = successful_results.nlargest(5, 'test_r2')
    for i, (_, row) in enumerate(top_5.iterrows(), 1):
        print(f"   {i}. {row['model_type']}-{row['backbone']}-{row['weighting']}-{row['loss_type']}")
        print(f"      Test R²: {row['test_r2']:.6f}, Train R²: {row['train_r2']:.6f}")
    
    # Component analysis
    components = ['model_type', 'backbone', 'weighting', 'loss_type']
    
    print(f"\n📈 COMPONENT CONTRIBUTION ANALYSIS:")
    for component in components:
        component_stats = successful_results.groupby(component)['test_r2'].agg(['mean', 'std', 'count'])
        print(f"\n   {component.upper()} PERFORMANCE:")
        for name, stats in component_stats.iterrows():
            print(f"      {name}: {stats['mean']:.4f}±{stats['std']:.4f} (n={stats['count']})")
    
    return successful_results

def create_thesis_summary_table(results_df):
    """Create publication-ready summary table"""
    
    successful_results = results_df[results_df['status'] == 'success'].copy()
    
    if len(successful_results) == 0:
        return None
    
    # Create summary statistics table
    summary_stats = []
    
    # By Model Type
    for model_type in successful_results['model_type'].unique():
        subset = successful_results[successful_results['model_type'] == model_type]
        summary_stats.append({
            'Category': 'Model Type',
            'Subcategory': model_type,
            'Mean_R2': subset['test_r2'].mean(),
            'Std_R2': subset['test_r2'].std(),
            'Best_R2': subset['test_r2'].max(),
            'Count': len(subset)
        })
    
    # By Backbone
    for backbone in successful_results['backbone'].unique():
        subset = successful_results[successful_results['backbone'] == backbone]
        summary_stats.append({
            'Category': 'GNN Backbone', 
            'Subcategory': backbone,
            'Mean_R2': subset['test_r2'].mean(),
            'Std_R2': subset['test_r2'].std(),
            'Best_R2': subset['test_r2'].max(),
            'Count': len(subset)
        })
    
    # By Weighting Scheme
    for weighting in successful_results['weighting'].unique():
        subset = successful_results[successful_results['weighting'] == weighting]
        summary_stats.append({
            'Category': 'Weighting Scheme',
            'Subcategory': weighting,
            'Mean_R2': subset['test_r2'].mean(),
            'Std_R2': subset['test_r2'].std(), 
            'Best_R2': subset['test_r2'].max(),
            'Count': len(subset)
        })
    
    # By Loss Type
    for loss_type in successful_results['loss_type'].unique():
        subset = successful_results[successful_results['loss_type'] == loss_type]
        summary_stats.append({
            'Category': 'Loss Function',
            'Subcategory': loss_type,
            'Mean_R2': subset['test_r2'].mean(),
            'Std_R2': subset['test_r2'].std(),
            'Best_R2': subset['test_r2'].max(),
            'Count': len(subset)
        })
    
    summary_df = pd.DataFrame(summary_stats)
    
    # Save summary table
    summary_df.to_csv('GNN_Thesis_Summary_Table.csv', index=False)
    
    print(f"\n📋 THESIS SUMMARY TABLE:")
    print("=" * 25)
    print(summary_df.round(4))
    
    return summary_df

def generate_thesis_insights(results_df):
    """Generate key insights for thesis discussion"""
    
    successful_results = results_df[results_df['status'] == 'success'].copy()
    
    if len(successful_results) == 0:
        return
    
    print(f"\n💡 KEY THESIS INSIGHTS:")
    print("=" * 25)
    
    # Best performing approaches
    best_config = successful_results.loc[successful_results['test_r2'].idxmax()]
    
    print(f"1. 🏆 BEST OVERALL PERFORMANCE:")
    print(f"   Configuration: {best_config['model_type']}-{best_config['backbone']}-{best_config['weighting']}")
    print(f"   Test R²: {best_config['test_r2']:.6f}")
    print(f"   Generalization Gap: {best_config['train_r2'] - best_config['test_r2']:.6f}")
    
    # Model type comparison
    model_comparison = successful_results.groupby('model_type')['test_r2'].mean()
    best_model_type = model_comparison.idxmax()
    
    print(f"\n2. 🏗️ MODEL ARCHITECTURE INSIGHTS:")
    print(f"   Best Model Type: {best_model_type}")
    print(f"   Performance Advantage: {model_comparison.max() - model_comparison.min():.6f}")
    
    # Backbone effectiveness
    backbone_comparison = successful_results.groupby('backbone')['test_r2'].mean()
    best_backbone = backbone_comparison.idxmax()
    
    print(f"\n3. 🧠 GNN BACKBONE INSIGHTS:")
    print(f"   Best Backbone: {best_backbone}")
    print(f"   Top 3 Backbones: {backbone_comparison.nlargest(3).index.tolist()}")
    
    # Weighting scheme effectiveness
    weighting_comparison = successful_results.groupby('weighting')['test_r2'].mean()
    best_weighting = weighting_comparison.idxmax()
    
    print(f"\n4. ⚖️ WEIGHTING SCHEME INSIGHTS:")
    print(f"   Best Weighting: {best_weighting}")
    print(f"   Performance Range: {weighting_comparison.max() - weighting_comparison.min():.6f}")
    
    # Loss function comparison
    loss_comparison = successful_results.groupby('loss_type')['test_r2'].mean()
    
    print(f"\n5. 🎯 LOSS FUNCTION INSIGHTS:")
    if len(loss_comparison) > 1:
        print(f"   Supervised vs Semi-supervised: {loss_comparison['supervised']:.6f} vs {loss_comparison.get('semi_supervised', 'N/A')}")
    
    # Generalization analysis
    generalization_gap = successful_results['train_r2'] - successful_results['test_r2']
    
    print(f"\n6. 📊 GENERALIZATION ANALYSIS:")
    print(f"   Mean Generalization Gap: {generalization_gap.mean():.6f}")
    print(f"   Std Generalization Gap: {generalization_gap.std():.6f}")
    print(f"   Best Generalizing Config: {successful_results.loc[generalization_gap.idxmin(), 'model_type']}-{successful_results.loc[generalization_gap.idxmin(), 'backbone']}")

# Run comprehensive analysis
if 'results_df' in locals() and len(results_df) > 0:
    print("🔬 Running comprehensive statistical analysis...")
    
    # Statistical analysis
    analyzed_results = comprehensive_statistical_analysis(results_df)
    
    # Thesis summary table  
    print(f"\n📋 Creating thesis-ready summary table...")
    summary_table = create_thesis_summary_table(results_df)
    
    # Key insights
    print(f"\n💡 Generating thesis insights...")
    generate_thesis_insights(results_df)
    
    print(f"\n🎯 FRAMEWORK CONTRIBUTIONS:")
    print("=" * 30)
    print("1. ✅ Novel GNN-GTWR/GTVC architectures implemented")
    print("2. ✅ Comprehensive comparison of 6 GNN backbones")
    print("3. ✅ 4 different spatial weighting schemes evaluated")
    print("4. ✅ Supervised vs semi-supervised loss comparison")
    print("5. ✅ Realistic spatial-temporal data simulation")
    print("6. ✅ Publication-quality visualizations generated")
    print("7. ✅ Statistical significance analysis completed")
    print("8. ✅ Thesis-ready results and insights provided")
    
else:
    print("⚠️ No results available for analysis")

print(f"\n🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊")
print(f"🏆 COMPREHENSIVE GNN-GTWR/GTVC FRAMEWORK COMPLETE!")
print(f"🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊🎊")

print(f"\n📚 FRAMEWORK SUMMARY:")
print(f"   🧠 GNN Backbones: {len(BACKBONE_REGISTRY)} implemented")
print(f"   ⚖️ Weighting Schemes: {len(WEIGHTING_REGISTRY)} implemented") 
print(f"   🏗️ Model Types: 2 (GNN-GTWR, GNN-GTVC)")
print(f"   🎯 Loss Functions: 3 (Supervised, Semi-supervised, Adversarial)")
print(f"   📊 Total Configurations: {len(BACKBONE_REGISTRY) * len(WEIGHTING_REGISTRY) * 2 * 2}")

print(f"\n🚀 READY FOR BAB4TA1.TEX INTEGRATION!")
print(f"📊 All results, visualizations, and analysis complete!")